##ResNet50 on Cifar Dataset

###Importing Libraries

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
%matplotlib inline
print("Version of tensorflow is : ",tf.__version__)

Version of tensorflow is :  2.11.0


###Getting the cifar data

In [ ]:
BATCH_SIZE = 32 
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
(training_images, training_labels) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 14s 0us/step


In [ ]:
training_images.shape, type(training_images), training_images.dtype

((50000, 32, 32, 3), numpy.ndarray, dtype('uint8'))

###Preprocessing

In [ ]:
def process_input(images):
    img = images.astype('float')
    img = tf.keras.applications.resnet50.preprocess_input(img)
    return img

In [ ]:
X_train = process_input(training_images)
X_val   = process_input(validation_images)

In [ ]:
X_train.shape, type(X_train), X_train.dtype

((50000, 32, 32, 3), numpy.ndarray, dtype('float64'))

In [ ]:
print("Shape of X_train : ",X_train.shape)
print("Shape of training_labels : ",training_labels.shape)
print("Shape of X_val : ",X_val.shape)
print("Shape of validation_labels : ",validation_labels.shape)

Shape of X_train :  (50000, 32, 32, 3)
Shape of training_labels :  (50000, 1)
Shape of X_val :  (10000, 32, 32, 3)
Shape of validation_labels :  (10000, 1)


###Defining the model

In [ ]:
def predefine_model(inputs):
    resnet = tf.keras.applications.resnet.ResNet50(weights='imagenet' , include_top=False , input_shape=(224,224,3))
    x = resnet(inputs)
    return x
     

def outputlayer(inputs):
    x = predefine_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024 , activation = 'relu')(x)
    x = tf.keras.layers.Dense(512  , activation = 'relu')(x)
    x = tf.keras.layers.Dense(10   , activation = 'softmax' , name="classification")(x)
    
    return x
     

def get_model():
    Input = tf.keras.layers.Input(shape=(32 , 32 , 3))
    x     = tf.keras.layers.UpSampling2D(size=(7,7))(Input)
    x     = outputlayer(x)

    model = tf.keras.models.Model(inputs = Input , outputs = x)
    model.compile(loss = tf.keras.losses.sparse_categorical_crossentropy , optimizer = 'sgd' , metrics = ['acc'])
    return model
     

In [ ]:
model = get_model()

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              209817

###Training the model

In [ ]:
epoch = 1
his = model.fit(X_train , training_labels ,
                epochs=epoch , batch_size=32 ,
                validation_data=(X_val , validation_labels) ,
                validation_batch_size=32)

1563/1563 [==============================] - 569s 349ms/step - loss: 0.3494 - acc: 0.8821 - val_loss: 0.1845 - val_acc: 0.9395


Able to get an validation accuracy of almost 94% in just 1 epoch